In [1]:
# automatically load changes from the sandbox_steps module
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import os
from e2b_desktop import Sandbox, CommandExitException
from dotenv import load_dotenv
load_dotenv()

sys.path.append(os.path.abspath('..')) # Add the parent directory (project root) to sys.path

from counter_strike.install_cs import install_cs_1_6, connect_to_server, choose_team
from counter_strike.image_handling import get_screenshot_message, draw_point, get_screenshot

from llms.models import AimingModel

E2B_API_KEY = os.environ.get("E2B_API_KEY")
CS_SERVER_IP = os.environ.get("CS_SERVER_IP")


In [3]:
# With custom configuration
desktop = Sandbox(
    display=":0",  # Custom display (defaults to :0)
    resolution=(1920, 1080),  # Custom resolution
    timeout = 3600) 

desktop.stream.start()

# Get stream URL
url = desktop.stream.get_url()
print(url)

# only viewing 
url_view = desktop.stream.get_url(view_only=True)
print(url_view)

https://6080-ipx7xikgaj16g4u1fqgzv-c65ff479.e2b.app/vnc.html?autoconnect=true&resize=scale
https://6080-ipx7xikgaj16g4u1fqgzv-c65ff479.e2b.app/vnc.html?autoconnect=true&view_only=true&resize=scale


In [184]:
install_cs_1_6(desktop=desktop)
connect_to_server(desktop=desktop, ip_address=CS_SERVER_IP)
choose_team(desktop=desktop)

Waiting for 150 secs for map download...


In [65]:
#desktop.write(text="ddddddddddddddddd", delay_in_ms=0, chunk_size=50)
desktop.write(text="dddddddddddddddddddd")
desktop.write(text="aaaaaaaaaaaaaaaaaaaa")
desktop.write(text="wwwwwwwwwwwwwwwwwwww")

In [3]:
desktop.kill()

In [4]:
from counter_strike.image_handling import get_mouse_movements
from counter_strike.controls import aim, shoot

from llms.models import AimingModel

aiming_model = AimingModel()

In [5]:
screenshot_message = get_screenshot_message(desktop)

In [6]:
aiming_model.complete(screenshot_message)

Model: qwen/qwen2.5-vl-32b-instruct
Provider: Fireworks


('```json\n{"point": {"x": "966", "y": "544"}}\n```',
 ChatCompletion(id='gen-1746964059-5JTwSxoL0nQbgLaUaWa9', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='```json\n{"point": {"x": "966", "y": "544"}}\n```', refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=None, reasoning=None), native_finish_reason='stop')], created=1746964059, model='qwen/qwen2.5-vl-32b-instruct', object='chat.completion', service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=24, prompt_tokens=2790, total_tokens=2814, completion_tokens_details=None, prompt_tokens_details=None), provider='Fireworks'))

In [11]:
from llms.tools import MoveTool
from llms.models import OpenRouterGameplayModel

move_tool = MoveTool(desktop=desktop)
tools = {move_tool.name: move_tool}

gameplay_model = OpenRouterGameplayModel(tools=tools, 
                                         model="google/gemini-2.5-flash-preview")

In [12]:
gameplay_model.complete(screenshot_message)

('',
 ChatCompletion(id='gen-1746964310-yJNS9m86uSZe2LkwuU2m', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content='', refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='tool_0_move_tool', function=Function(arguments='{"key_sequence":"wwasd"}', name='move_tool'), type='function', index=0)], reasoning=None), native_finish_reason='STOP')], created=1746964310, model='google/gemini-2.5-flash-preview', object='chat.completion', service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=8, prompt_tokens=2042, total_tokens=2050, completion_tokens_details=None, prompt_tokens_details=None), provider='Google'),
 [ChatCompletionMessageToolCall(id='tool_0_move_tool', function=Function(arguments='{"key_sequence":"wwasd"}', name='move_tool'), type='function', index=0)])

In [208]:
from counter_strike.agent import run_agent

run_agent(
    aiming_model=aiming_model,
    gameplay_model=gameplay_model,
    desktop=desktop,
    iterations=20 # For demonstration
) 

ImageLoggingSettings: Session directory created at ../images\20250510_214313

--- Iteration 1 ---
  [Time] Screenshot: 1.9487s
Model: qwen/qwen2.5-vl-32b-instruct
Provider: Fireworks
  [Time] Aiming Model: 4.4648s
  [Action] No Coords. Using Gameplay Model Tool Calls.
  [Time] Gameplay Model: 0.1749s
 Action taken: Tool Calls: [ChatCompletionMessageToolCall(id='tool_0_move_tool', function=Function(arguments='{"key_sequence":"wwwwd"}', name='move_tool'), type='function', index=0)]
  [Time] Iteration 1 Total: 9.1135s

--- Iteration 2 ---
  [Time] Screenshot: 1.9071s
Model: qwen/qwen2.5-vl-32b-instruct
Provider: Fireworks


KeyboardInterrupt: 

In [ ]:
import concurrent.futures

for i in range(10):
    print(f"Iteration {i+1}") # Changed None to i+1 for clarity
    screenshot_message = get_screenshot_message(desktop, filename="../images/screenshot.jpg")

    coords = None
    point_json = None
    content = None
    tool_calls = None
    aiming_done_first_and_valid = False

    with concurrent.futures.ThreadPoolExecutor(max_workers=2) as executor:
        future_aiming = executor.submit(aiming_model.complete, user_messages=screenshot_message, debug=True)
        future_gameplay = executor.submit(gameplay_model.complete, user_messages=screenshot_message)

        futures = [future_aiming, future_gameplay]
        future_to_task = {future_aiming: "aiming", future_gameplay: "gameplay"}

        try:
            point_json_result, _ = future_aiming.result(timeout=None) 
            print(f"Aiming model output: {point_json_result}")
            parsed_coords = aiming_model.parse_point_json(point_json_result)
            if parsed_coords:
                coords = parsed_coords
                aiming_done_first_and_valid = True
                # If coords are found, we can cancel the gameplay future if it's not needed
                # and not already running or completed.
                if not future_gameplay.done():
                    future_gameplay.cancel()
                else: # if it's already done, get its result to avoid unretrieved exception
                    try:
                        content_result, response, tool_calls_result = future_gameplay.result(timeout=0.1)
                        content = content_result
                        tool_calls = tool_calls_result
                        print("Gameplay model completed even though coords were found.")
                    except (concurrent.futures.TimeoutError, concurrent.futures.CancelledError):
                        print("Gameplay model was cancelled or timed out after aiming was successful.")
                    except Exception as e:
                        print(f"Error retrieving result from already done gameplay future: {e}")


        except Exception as e:
            print(f"Error in aiming_model.complete: {e}")

        # If coords were found and processed, we skip waiting for gameplay if it was cancelled
        if aiming_done_first_and_valid:
            pass # Handled above
        else:
            # If coords were not found from aiming_model (either it failed or returned no coords)
            # we now need the result from gameplay_model
            print("No valid coords from aiming model, waiting for gameplay model.")
            try:
                # Wait for the gameplay future to complete if it hasn't been processed
                if not future_gameplay.done() or not tool_calls: # check if tool_calls already populated
                    content_result, _, tool_calls_result = future_gameplay.result(timeout=None) # Wait indefinitely
                    content = content_result
                    tool_calls = tool_calls_result
                    print(f"Gameplay model output: content='{content}', tool_calls='{tool_calls}'")
            except concurrent.futures.CancelledError:
                print("Gameplay model task was cancelled.")
            except Exception as e:
                print(f"Error in gameplay_model.complete: {e}")


    # Logic based on the results
    if coords:
        print(f"Coordinates found: {coords}. Proceeding with aiming and shooting.")
        draw_point(point=coords, image_path="../images/screenshot.jpg", output_path="../images/screenshot_annotated.jpg")
        mouse_movements = get_mouse_movements(coords=coords)
        aim(mouse_movements, desktop=desktop)
        shoot(desktop=desktop)
    elif tool_calls: # If no coords, but we have tool_calls from the gameplay model
        print(f"No coordinates. Using tool_calls: {tool_calls}")
        gameplay_model._handle_tool_calls(tool_calls=tool_calls)
    else:
        print(f"The gameplay model did not return any tool calls. Response.") # print the response here.

    print("-" * 30) # Separator for iterations

Iteration 1
[{'role': 'user', 'content': 'Current map: aim_map_2010.Design five consecutive movement steps to locate the enemy: 1) Choose a combined sequence of 5 keys (w/a/s/d) per move. 2) Return your answer by calling move_tool with the \'key_sequence\' parameter. Example: {"name": "move_tool", "arguments": {"key_sequence": "wwaad"}}'}]
ChatCompletion(id='gen-1746896490-iwwebrGcnOAtNf09cx0H', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='```json\n{"point": {"x": "1065", "y": "542"}}\n```', refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=None, reasoning=None), native_finish_reason='stop')], created=1746896490, model='qwen/qwen2.5-vl-72b-instruct', object='chat.completion', service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=25, prompt_tokens=2790, total_tokens=2815, completion_tokens_details=None, prompt_tokens_details=None), provider='Parasail')
Aiming 

KeyboardInterrupt: 